## AutoMobile Prediction 

###### CHINTAN CHITRODA

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('/kaggle/input/automobile-dataset/AutoData.csv')

In [ ]:
df.head(5)

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

### EDA

In [ ]:
plt.subplots(figsize=(15,10))
sns_plot = sns.heatmap(df.corr(),annot =True,cmap='rainbow_r',square=True)
fig = sns_plot.get_figure()
fig.savefig("corrHeatmap.png")

In [ ]:
tempdf = df[{'symboling','enginetype','boreratio','horsepower', 'peakrpm', 'citympg', 'highwaympg','enginesize','price'}]

In [ ]:
sns.pairplot(tempdf)

In [ ]:
for i in df.columns:
    plt.title(i)
    sns.scatterplot(df[i],df['price'])
    plt.show()

In [ ]:
bestvar1 = ['wheelbase','carlength','carwidth','curbweigth','enginesize']

In [ ]:
sns.distplot(df['price'])

###### we see there are cars highest in price range below !0000 $

## Data PreProcessing

In [ ]:
df.columns

In [ ]:
df1 = df.drop({'make','carlength','carwidth','aspiration', 'doornumber','carbody','drivewheel','carheight',"enginelocation"},axis=1)

In [ ]:
df1.columns

In [ ]:
df1

In [ ]:
from sklearn import preprocessing 
label_encoder = preprocessing.LabelEncoder() 
df1['fueltype']= label_encoder.fit_transform(df['fueltype']) 
df1['fueltype'].unique()

In [ ]:
df1

In [ ]:
df1['enginetype']= label_encoder.fit_transform(df['enginetype']) 
df1['enginetype'].unique()

In [ ]:
df1['fuelsystem']= label_encoder.fit_transform(df['fuelsystem']) 
df1['fuelsystem'].unique()

In [ ]:
from word2number import w2n

In [ ]:
nums = []
for i in df1['cylindernumber']:
    x = w2n.word_to_num(i)
    nums.append(x)
df1['cylindernumber'] = nums

In [ ]:
df1

##  Linear Regression

#### Self Analysis based feature

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import numpy as np
import statsmodels.api as sm

In [ ]:
X = df1[['horsepower']]#.values.reshape(1,-1)
y = df1[['price']]#.values.reshape(1,-1)

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=.7,random_state=42)

In [ ]:
LR = LinearRegression()

In [ ]:
LR.fit(X_train,y_train)

In [ ]:
y_pred = LR.predict(X_test)

In [ ]:
r2_score(y_pred,y_test)

In [ ]:
print('Coefficient is',LR.coef_[0][0])
print('intercept is',LR.intercept_[0])

In [ ]:
g = sns.jointplot(X,y,kind='reg',
                 joint_kws={'line_kws':{'color':'red'}})
plt.xlabel('HorsePower')
plt.ylabel('Price')

In [ ]:
plt.scatter(X,y)
plt.plot(X,LR.predict(X))
plt.xlabel('HorsePower')
plt.ylabel('Price')
plt.show

In [ ]:
model = sm.OLS(y, X)
results = model.fit()
print(results.summary())

###### We see the Regression LIne is in Best Fit

#### RFE selected Feature

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.preprocessing import MinMaxScaler

In [ ]:
X = df1.drop('price',axis=1)
y = df1['price']
X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=.7,random_state=42)

In [ ]:
scaler = MinMaxScaler()
X_train1 = scaler.fit_transform(X_train)

In [ ]:
model = LinearRegression()

In [ ]:
rfe = RFE(model, 1)

In [ ]:
rfe.fit(X_train,y_train)

In [ ]:
X_train.columns[rfe.support_]

In [ ]:
## Model Building
X = df[['boreratio']]
y = df[['price']]
X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=.7,random_state=42)
LR = LinearRegression()
LR.fit(X_train,y_train)
y_pred1 = LR.predict(X_train)
print("R2 score is",r2_score(y_pred,y_test))
print('Coefficient is',LR.coef_[0][0])
print('intercept is',LR.intercept_[0])

In [ ]:
plt.scatter(X,y)
plt.plot(X,LR.predict(X))
plt.xlabel('Boreratio')
plt.ylabel('Price')
plt.show

In [ ]:
g = sns.jointplot(X,y,kind='reg',
                 joint_kws={'line_kws':{'color':'red'}})
plt.xlabel('Boreration')
plt.ylabel('Price')

In [ ]:
model = sm.OLS(y, X)
results = model.fit()
print(results.summary())

## Multiple Linear Regression

#### Self Analysis based Features

In [ ]:
df1.columns

In [ ]:
X = df1.drop('price',axis=1)
y = df1['price']

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=.7,random_state=42)

In [ ]:
LR.fit(X_train.values,y_train)

In [ ]:
y_pred = LR.predict(X_test)

In [ ]:
r2_score(y_pred,y_test)

In [ ]:
print('Coefficient is',LR.coef_)
print('intercept is',LR.intercept_)

In [ ]:
model = sm.OLS(y, X)
results = model.fit()
print(results.summary())

#### RFE Based features

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler = MinMaxScaler()
model = LinearRegression()

In [ ]:
## Selecting Top 7 Features
rfe = RFE(model, 7)
X_train1 = scaler.fit_transform(X_train)

In [ ]:
rfe.fit(X_train1,y_train)

In [ ]:
X_train.columns[rfe.support_]

In [ ]:
print("Num Features: %s" % (rfe.n_features_))
print("Selected Features: %s" % (rfe.support_))
print("Feature Ranking: %s" % (rfe.ranking_))

In [ ]:
#X = df1[{'symboling', 'fueltype', 'boreratio', 'compressionratio', 'citympg'}]
X = df1[{'wheelbase', 'enginesize', 'stroke', 'compressionratio', 'horsepower',
       'peakrpm', 'citympg'}]
y = df1['price']
X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=.7,random_state=42)

In [ ]:
LR = LinearRegression()
LR.fit(X_train,y_train)
y_pred1 = LR.predict(X_train)
print("R2 score is",r2_score(y_pred,y_test))
print('intercept is',LR.intercept_)
print('Coefficient are',LR.coef_)

# The End
### Please do upvote if you like ¯\_(ツ)_/¯